# Data Science Project

* Name: Author Name
* Email:


## TABLE OF CONTENTS 


- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

**This note book takes 3 minutes to run**

# INTRODUCTION

## Imports



In [1]:
# Importing packages
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import shutil
import os
import sqlite3
import db_to_sqlite
from sqlite3 import Error
import csv
from pathlib import Path
import subprocess
import io
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
from functions_all import *


%reload_ext autoreload
%autoreload 2
%matplotlib inline

# OBTAIN

## Data

### Data source

Data is from FBI Crime Data Explorer
[NIBRS data for Colorado from 2009-2019](https://crime-data-explorer.fr.cloud.gov/pages/downloads)

The [data dictionary](data/NIBRS_DataDictionary.pdf) is  and a [record descriptiopn](data/NIBRS_Record_Description.pdf) are available.


The description of the main and reference tables is in data/README.md file.
The agency implemented some changes to the files structure in 2016 and removed the sqlite create and load scripts from the zip directories.
Another fact worth mentioning is that files 'nibrs_property_desc.csv' from 2014 and 2015 have duplicated nibrs_property_desc_ids (unique identifier in the nibrs_property_desc table) which complicated the loading of the data.

### Loading it all up into SQLite database for easy access

**Before re-running the code if needed remove the existing database** 

In [2]:
# cur.close()
# conn.close()

In [3]:
!rm data/sqlite/db/production1.db

In [4]:
#I created a separate directory with only incident data files as a template for lists of data (not reference tables)
# from 2009-2015

list_template_early=os.listdir('data/incidents/template_data/')
list_template_early

['agency_participation.csv',
 'cde_agencies.csv',
 'nibrs_arrestee.csv',
 'nibrs_arrestee_weapon.csv',
 'nibrs_bias_motivation.csv',
 'nibrs_criminal_act.csv',
 'nibrs_incident.csv',
 'nibrs_month.csv',
 'nibrs_offender.csv',
 'nibrs_offense.csv',
 'nibrs_property.csv',
 'nibrs_property_desc.csv',
 'nibrs_suspected_drug.csv',
 'nibrs_suspect_using.csv',
 'nibrs_victim.csv',
 'nibrs_victim_circumstances.csv',
 'nibrs_victim_injury.csv',
 'nibrs_victim_offender_rel.csv',
 'nibrs_victim_offense.csv',
 'nibrs_weapon.csv']

In [5]:
# List of incident data fiels from 2016-2019

list_template_late=list_template_early[2:]
list_template_late.append('agencies.csv')
list_template_late

['nibrs_arrestee.csv',
 'nibrs_arrestee_weapon.csv',
 'nibrs_bias_motivation.csv',
 'nibrs_criminal_act.csv',
 'nibrs_incident.csv',
 'nibrs_month.csv',
 'nibrs_offender.csv',
 'nibrs_offense.csv',
 'nibrs_property.csv',
 'nibrs_property_desc.csv',
 'nibrs_suspected_drug.csv',
 'nibrs_suspect_using.csv',
 'nibrs_victim.csv',
 'nibrs_victim_circumstances.csv',
 'nibrs_victim_injury.csv',
 'nibrs_victim_offender_rel.csv',
 'nibrs_victim_offense.csv',
 'nibrs_weapon.csv',
 'agencies.csv']

> I commented out the following cell to avoid overwriting changes to the directories

In [6]:
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2009/', 'data/incidents/2009/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2010/', 'data/incidents/2010/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2011/', 'data/incidents/2011/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2012/', 'data/incidents/2012/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2013/', 'data/incidents/2013/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2014/', 'data/incidents/2014/')
# copy_files(list_template_early, '/Users/elena/Desktop/FBI_crime_files/CO-2015/', 'data/incidents/2015/')

In [7]:
# copy_files(list_template_late, '/Users/elena/Desktop/FBI_crime_files/CO-2016/', 'data/incidents/2016/')
# copy_files(list_template_late, '/Users/elena/Desktop/FBI_crime_files/CO-2017/', 'data/incidents/2017/')
# copy_files(list_template_late, '/Users/elena/Desktop/FBI_crime_files/CO-2018/', 'data/incidents/2018/')
# copy_files(list_template_late, '/Users/elena/Desktop/FBI_crime_files/CO-2019/', 'data/incidents/2019/')

In [8]:
# Initiating a cursor
conn = sqlite3.connect('data/sqlite/db/production1.db')
cur = conn.cursor()

In [9]:
q="SELECT name FROM sqlite_master WHERE type='table'"
df=table_query(q, cur=cur)
df

Nothing was found


In [10]:
# Creating tables in the database
sql_file = open('script_to_create_tables.sql')
sql_as_string = sql_file.read()
cur.executescript(sql_as_string)


In [11]:
q="SELECT name FROM sqlite_master WHERE type='table'"
df=table_query(q, cur=cur)
df

,name
0,agencies
1,agency_participation
2,cde_agencies
3,nibrs_activity_type
4,nibrs_age
5,nibrs_arrest_type
6,nibrs_assignment_type
7,nibrs_bias_list
8,nibrs_location_type
9,nibrs_offense_type


In [12]:
display_csvfileDF('nibrs_age.csv', 'Ref_tables/')

,age_id,age_code,age_name
0,1,NN,Under 24 Hours
1,2,NB,1-6 Days Old
2,3,BB,7-364 Days Old
3,4,00,Unknown
4,5,AG,Age in Years
5,6,99,Over 98 Years Old


In [13]:
q="""SELECT * FROM nibrs_age"""
df=table_query(q, cur=cur)
df

Nothing was found


In [14]:
 #All reference table files are in this directory, the actual incident data files are in all data/incidents, split by years
!ls -al data/Ref_tables/

total 56
drwxr-xr-x 1 elena 197121    0 Jun 30 14:49 .
drwxr-xr-x 1 elena 197121    0 Jul 14 00:47 ..
-rw-r--r-- 1 elena 197121  477 Jun 30 14:44 nibrs_activity_type.csv
-rw-r--r-- 1 elena 197121  137 Jun 30 14:44 nibrs_age.csv
-rw-r--r-- 1 elena 197121  105 Jun 30 14:44 nibrs_arrest_type.csv
-rw-r--r-- 1 elena 197121  266 Jun 30 14:44 nibrs_assignment_type.csv
-rw-r--r-- 1 elena 197121  993 Jun 30 14:44 nibrs_bias_list.csv
-rw-r--r-- 1 elena 197121  556 Jun 30 14:44 nibrs_circumstances.csv
-rw-r--r-- 1 elena 197121  217 Jun 30 14:44 nibrs_cleared_except.csv
-rw-r--r-- 1 elena 197121  442 Jun 30 14:44 nibrs_criminal_act_type.csv
-rw-r--r-- 1 elena 197121  218 Jun 30 14:44 nibrs_drug_measure_type.csv
-rw-r--r-- 1 elena 197121  134 Jun 30 14:44 nibrs_ethnicity.csv
-rw-r--r-- 1 elena 197121  194 Jun 30 14:44 nibrs_injury.csv
-rw-r--r-- 1 elena 197121  436 Jun 30 14:44 nibrs_justifiable_force.csv
-rw-r--r-- 1 elena 197121 1238 Jun 30 14:44 nibrs_location_type.csv
-rw-r--r-- 1 elena 197121 

In [15]:
# Creating a list of ref table files to import them into tables
files_ref=create_filelist('data/Ref_tables/',n=0)
files_ref

['data/Ref_tables/nibrs_activity_type.csv',
 'data/Ref_tables/nibrs_age.csv',
 'data/Ref_tables/nibrs_arrest_type.csv',
 'data/Ref_tables/nibrs_assignment_type.csv',
 'data/Ref_tables/nibrs_bias_list.csv',
 'data/Ref_tables/nibrs_circumstances.csv',
 'data/Ref_tables/nibrs_cleared_except.csv',
 'data/Ref_tables/nibrs_criminal_act_type.csv',
 'data/Ref_tables/nibrs_drug_measure_type.csv',
 'data/Ref_tables/nibrs_ethnicity.csv',
 'data/Ref_tables/nibrs_injury.csv',
 'data/Ref_tables/nibrs_justifiable_force.csv',
 'data/Ref_tables/nibrs_location_type.csv',
 'data/Ref_tables/nibrs_offense_type.csv',
 'data/Ref_tables/nibrs_prop_desc_type.csv',
 'data/Ref_tables/nibrs_prop_loss_type.csv',
 'data/Ref_tables/nibrs_relationship.csv',
 'data/Ref_tables/nibrs_suspected_drug_type.csv',
 'data/Ref_tables/nibrs_using_list.csv',
 'data/Ref_tables/nibrs_victim_type.csv',
 'data/Ref_tables/nibrs_weapon_type.csv',
 'data/Ref_tables/ref_race.csv',
 'data/Ref_tables/ref_state.csv']

In [16]:
#importing data into reference tables
import_data_to_tables('data/sqlite/db/production1.db', files_ref, 'data/Ref_tables/')

In [17]:
q='SELECT * FROM nibrs_using_list'
df=table_query(q, cur)
df

,suspect_using_id,suspect_using_code,suspect_using_name
0,1,A,Alcohol
1,2,C,Computer Equipment
2,3,D,Drugs/Narcotics
3,4,N,Not Applicable


In [18]:
# Importing incidents data from 2009-2015 to the database

list_inc_2009=create_filelist('data/incidents/2009/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2009, 'data/incidents/2009/')

list_inc_2010=create_filelist('data/incidents/2010/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2010, 'data/incidents/2010/')

list_inc_2011=create_filelist('data/incidents/2011/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2011, 'data/incidents/2011/')

list_inc_2012=create_filelist('data/incidents/2012/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2012, 'data/incidents/2012/')

list_inc_2013=create_filelist('data/incidents/2013/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2013, 'data/incidents/2013/')

list_inc_2014=create_filelist('data/incidents/2014/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2014, 'data/incidents/2014/')

list_inc_2015=create_filelist('data/incidents/2015/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2015, 'data/incidents/2015/')

In [19]:
q='SELECT * FROM nibrs_incident'
df=table_query(q,cur)
len(df)

1701394

**All 2016-2019 files need to be cleaned up because FBI changed the file format. There is a YEAR column that needs to be removed as well as the legacy columns from the previous years need to be added up. It's a tedious job and it needs to be done once and the files need to be backed up.**

In order to clean the tables up the following needs to be done<br>

   1. Remove all **DATA_YEAR** columns from each file, it's the first column<br>
   
   2. Files that do not need any changes beyond **DATA_YEAR** column removal<br>
    
> nibrs_arrestee_weapon.csv<br>
nibrs_bias_motivation.csv<br>
nibrs_criminal_act.csv<br>
nibrs_property_desc.csv<br>
nibrs_suspect_using.csv<br>
nibrs_suspected_drug.csv<br>
nibrs_victim_circumstances.csv<br>
nibrs_victim_injury.csv<br>
nibrs_victim_offender_rel.csv<br>
nibrs_victim_offense.csv<br>
nibrs_weapon.csv<br>

    
   3. in **nibrs_arestee.csv file**:<br><br>
   a. between **ARRESTEE_SEQ_NUM** and **ARREST_DATE** there should be an **arrest_num column**<br>
   b. Between **CLEARANCE_IND** and **AGE_RANGE_LOW_NUM** should be a **ff_line_number** column. <br>

4.  in **nibrs_incident** file:<br><br>
    a.between **NIBRS_MONTH_ID** and **CARGO_THEFT_FLAG** column **incident_number**<br>
    b.between **DATA_HOME** and **ORIG_FORMAT** column **ddocname**<br>
    c.between **ORIG_FORMAT** and **DID** column	**ff_line_number**<br><br>

5. in **nibrs_month.csv** file:<br><br>
    a.between **REPORT_DATE** and **UPDATE_FLAG** add **prepared_date** column<br>
    b.between **ORIG_FORMAT** and **DATA_HOME** column **ff_line_number**<br>
    c.column **MONTH_PUB_STATUS** removed<br><br>

6. in **nibrs_offender.csv** file:<br><br>
     a.between **ETHNICITY_ID** and **AGE_RANGE_LOW_NUM** column **ff_line_number**<br><br>
     
7. in **nibrs_offense.csv** file:<br><br>
     a. the last column **ff_line_number** should be added<br><br>
   
8. in **nibrs_property.csv** file:<br><br>
     a. the last column **ff_line_number** should be added<br><br>

9. in **nibrs_victim.csv** file:<br><br>
     a. between **RESIDENT_STATUS_CODE** and **AGE_RANGE_LOW_NUM** two columns **agency_data_year** and **ff_line_number** (in that order) should be added
    


In [20]:
# Importing cleaned-up files to the tables and checking numbers along the way

list_inc_2016=create_filelist('data/incidents/2016/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2016, 'data/incidents/2016/')

In [21]:
q='SELECT * FROM nibrs_incident'
df=table_query(q,cur)
len(df)

1983733

In [22]:
list_inc_2017=create_filelist('data/incidents/2017/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2017, 'data/incidents/2017/')

In [23]:
q='SELECT * FROM nibrs_incident'
df=table_query(q, cur)
len(df)

2269247

In [24]:
list_inc_2018=create_filelist('data/incidents/2018/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2018, 'data/incidents/2018/')

In [25]:
q='SELECT * FROM nibrs_incident'
df=table_query(q, cur)
len(df)

2556043

In [26]:
list_inc_2019=create_filelist('data/incidents/2019/', n=0)
import_data_to_tables('data/sqlite/db/production1.db', list_inc_2019, 'data/incidents/2019/')

In [27]:
q='SELECT * FROM nibrs_incident'
df=table_query(q, cur)
len(df)

2819463

In [28]:
df.head()

,agency_id,incident_id,nibrs_month_id,incident_number,cargo_theft_flag,submission_date,incident_date,report_date_flag,incident_hour,cleared_except_id,cleared_except_date,incident_status,data_home,ddocname,orig_format,ff_line_number,did
0,1971,51264520,4814762,09000019,,,2009-01-05 00:00:00,,22,6,,0,C,2009_01_CO0320000_09000019_INC_NIBRS,,,
1,1971,51264521,4814762,09000053,,,2009-01-13 00:00:00,,,6,,0,C,2009_01_CO0320000_09000053_INC_NIBRS,,,
2,1971,51264523,4814762,09000082,,,2009-01-17 00:00:00,,19,6,,0,C,2009_01_CO0320000_09000082_INC_NIBRS,,,
3,1971,51264524,4814762,09000092,,,2009-01-20 00:00:00,R,,6,,0,C,2009_01_CO0320000_09000092_INC_NIBRS,,,
4,1971,51264525,4814762,09000097,,,2009-01-21 00:00:00,,,6,,0,C,2009_01_CO0320000_09000097_INC_NIBRS,,,


In [29]:
cur.close()
conn.commit()
conn.close()

## This notebook is the pre-work for the notebook, part I

The link to the main notebook [here](capstone_prj_v1.0.ipynb)

## All the tables from 2009-2019 incidents in Colorado is in production1 database in data/sqlite/db folder

> This database can be used moving forward